In [1]:
import numpy as np
import pandas as pd
import eipy.ei as e
import tensorflow as tf
import pickle as pkl
from skimage.transform import resize
from importlib import reload

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.additional_ensembles import MeanAggregation, CES
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

2024-01-31 20:31:17.819469: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 20:31:17.849043: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 20:31:17.849069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 20:31:17.849853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 20:31:17.854610: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-31 20:31:17.855569: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

In [3]:
from keras import layers, models
from keras.applications import ResNet50, MobileNet, InceptionV3, DenseNet121

res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# base_model = MobileNet(weights='imagenet', include_top=False)
# x = base_model.output
# for layer in base_model.layers:
#     layer.trainable = False
# x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dense(1024, activation='relu')(x)
# predictions = layers.Dense(1, activation='sigmoid')(x)
# net_model = models.Model(inputs=base_model.input, outputs=predictions)
# net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# pretrained_model = InceptionV3(weights='imagenet', include_top=False)
# for layer in pretrained_model.layers:
#     layer.trainable = False
# collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
# output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
# v3_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
# v3_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# dense_model = DenseNet121(weights='imagenet', include_top=False)
# for layer in pretrained_model.layers:
#     layer.trainable = False
# collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
# output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
# dense121_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
# dense121_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

base_predictors = {"ResNet" : res_model,
                #    "MobileNet" : net_model,
                #    "Inception" : v3_model,
                #    "DenseNet" : dense121_model
}

In [4]:
metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
train_size = 100
X_train, y_train = X_train[:train_size], y_train[:train_size]
X_test, y_test = X_test[train_size:train_size+int(train_size/10)], y_test[train_size:train_size+int(train_size/10)]

In [7]:
inanimate_labels=[0,1,8,9]
def binarize_array(arr):
    mask = np.isin(arr, inanimate_labels)

    arr = np.where(mask, 0, 1)

    return arr
y_train = binarize_array(y_train)
y_test = binarize_array(y_test)

In [8]:
new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')

In [36]:
####HI OVER HERE###
reload(e)
###HEYOOOOOO#

<module 'eipy.ei' from '/home/opc/eipy/eipy/ei.py'>

In [37]:
EI = e.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=2,
                        sampling_strategy="hybrid",
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="dl",
                        model_building=True,
                        )

In [38]:
EI.fit_base(X_train_resized, y_train, modality_name="images")

Training base predictors on images...
        
... for ensemble performance analysis...
1/1 [==============================] - 0s 234ms/step

... for final ensemble...


Generating final ensemble training data: |          |  0%

4/4 [==============================] - 1s 250ms/step - loss: 0.2423 - accuracy: 0.9600


Generating final ensemble training data: |██████████|100%

In [39]:
EI.base_summary["metrics"]

modality,images
base predictor,ResNet
f_max,0.948718
auc,0.977929
mcc,0.894625


In [40]:
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

Analyzing ensembles: |██████████|100%
Training final ensemble models: |██████████|100%


In [41]:
EI.ensemble_summary["metrics"]

,Mean,CES,S.ADAB,S.XGB,S.DT,S.RF,S.GB,S.KNN,S.LR,S.NB,S.MLP,S.SVM
f_max,0.948718,0.948718,0.921053,0.938272,0.921053,0.950000,0.921053,0.926829,0.948718,0.950000,0.948718,0.938272
auc,0.977929,0.977929,0.944610,0.971986,0.936333,0.973896,0.950127,0.970501,0.977929,0.973260,0.977929,0.967742
mcc,0.894625,0.894625,0.872666,0.901362,0.872666,0.872666,0.872666,0.874722,0.872666,0.872666,0.872666,0.894625


In [42]:
X_test_dict = {"images": X_test_resized}
y_pred = EI.predict(X_test_dict, ensemble_model_key="S.LR")

1/1 [==============================] - 1s 713ms/step
TRUE
1/1 [==============================] - 1s 692ms/step
TRUE


In [45]:
from sklearn.metrics import accuracy_score
y_pred = [y > 0.5 for y in y_pred]
accuracy_score(y_pred, y_test)

0.9